In [1]:
import torch
import json

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.agents import create_agent
from langchain.tools import tool

from client.object_detection import DetectionClient
from client.segmentation import SegmentationClient
from client.generation import ImageGenerationClient # generation.py에 있다고 가정

c:\Users\User\miniproject\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BACKEND_URL = "http://157.66.255.80:14252"
VLLM_BASE_URL=BACKEND_URL + "/vllm/v1"
IMAGE_BASE_URL=BACKEND_URL + "/image"

detection_client = DetectionClient(base_url=IMAGE_BASE_URL)
segmentation_client = SegmentationClient(base_url=IMAGE_BASE_URL)
inpainting_client = ImageGenerationClient(base_url=IMAGE_BASE_URL)

In [3]:
@tool
def detect_object(image_path: str, object_name: str) -> str:
    """
    이미지에서 특정 객체('object_name')의 위치를 찾을 때 사용합니다.
    입력: 이미지 파일 경로('image_path'), 찾을 객체의 이름('object_name').
    출력: 찾은 객체의 bounding box 좌표를 담은 JSON 문자열.
    """
    print(f"--- 1. [실제 API 호출] 객체 탐지 실행: {object_name} ---")
    
    try:
        # 1. 이미지 파일을 열어서 바이트(bytes) 데이터로 읽습니다.
        with open(image_path, "rb") as image_file:
            image_bytes = image_file.read()

        # 2. detection_client.detect 메소드를 호출합니다.
        #    이때, 이 파일(DetectionClient)의 규칙에 맞춰 인자를 전달합니다.
        result_obj = detection_client.detect(
            image=image_bytes,                # 이미지 데이터
            text=[object_name],               # <<<--- 1. 문자열을 리스트로 감싸서 전달!
            threshold=0.25                    # 기본값 사용
        )

        # 3. 백엔드로부터 받은 결과 객체(DetectorOutput)를 딕셔너리로 변환 후,
        #    JSON 문자열로 만들어 반환합니다.
        return result_obj.model_dump_json()

    except Exception as e:
        # API 호출 중 오류가 발생하면, 에이전트에게 오류 내용을 알려줍니다.
        return f"Detect API 호출 중 오류 발생: {e}"

In [4]:
@tool
def segment_object(image_path: str, bounding_box_json: str) -> str:
    """이미지에서 주어진 좌표에 해당하는 영역의 마스크를 생성하고 마스크가 저장된 경로를 리턴합니다."""
    print(f"\n--- 2. [도구 실행] segment_object ---")
    print(f"   - 입력: image_path='{image_path}', bounding_box_json='{bounding_box_json}'")

    try:
        # 1. json.loads()를 통해 문자열을 파이썬 리스트(float 포함)로 변환합니다.
        box_data_float = json.loads(bounding_box_json)
        
        # 2. 리스트의 모든 항목을 정수(integer)로 변환합니다.
        #    예: [144.73, 255.1] -> [144, 255]
        # <<<--- 여기가 가장 중요한 수정 포인트! ---
        box_data_int = [int(coord) for coord in box_data_float]

        with open(image_path, "rb") as image_file:
            image_bytes = image_file.read()
        
        # 3. API 요청 시 정수로 변환된 리스트를 전달합니다.
        print(f"   - API 요청: box={box_data_int} 를 /segment/box 엔드포인트로 보냅니다.")
        
        segmentation_result = segmentation_client.segment_box(
            image=image_bytes, 
            box=box_data_int  # 정수형 리스트를 전달
        )
        
        # (이하 코드는 동일)
        mask = segmentation_result.extract_masks()[0]
        mask_path = "mask_result.png"
        mask.save(mask_path)
            
        print(f"   - API 응답 (성공): 마스크 파일 저장 완료 -> '{mask_path}'")
        return mask_path

    except Exception as e:
        print(f"   - API 호출 오류: {e}")
        return f"Segment API 호출 중 오류 발생: {e}"

In [6]:
@tool
def inpaint_image(image_path: str, mask_path: str, prompt: str) -> str:
    """원본 이미지의 마스크 영역을 주어진 프롬프트에 따라 새롭게 채우고 저장된 이미지 경로를 리턴합니다."""
    print(f"\n--- 3. [도구 실행] inpaint_image ---")
    print(f"   - 입력: image_path='{image_path}', mask_path='{mask_path}', prompt='{prompt}'")

    try:
        # 1. 원본 이미지와 마스크 이미지 파일을 각각 열어서 바이트 데이터로 읽습니다.
        with open(image_path, "rb") as image_file, open(mask_path, "rb") as mask_file:
            image_bytes = image_file.read()
            mask_bytes = mask_file.read()
            
        print(f"   - API 요청: prompt='{prompt}' 를 /generate/inpaint 엔드포인트로 보냅니다.")
        
        # 2. inpainting_client의 'inpaint' 메소드를 호출합니다.
        #    이때, Client의 규칙에 맞는 'control_image', 'control_mask' 매개변수 이름을 사용합니다.
        # <<<--- 여기가 가장 중요한 수정 포인트! ---
        result_image_object = inpainting_client.inpaint(
            control_image=image_bytes,
            control_mask=mask_bytes,
            prompt=prompt
        )
        
        # 3. 백엔드로부터 받은 PIL.Image 객체를 실제 파일로 저장합니다.
        #    바이트 데이터가 아니므로 .save() 메소드를 사용해야 합니다.
        result_path = "final_result.png"
        result_image_object.save(result_path)
            
        print(f"   - API 응답 (성공): 최종 이미지 저장 완료 -> '{result_path}'")
        return result_path
        
    except Exception as e:
        print(f"   - API 호출 오류: {e}")
        return f"Inpaint API 호출 중 오류 발생: {e}"

In [7]:
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.tools import tool
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

In [8]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model="model",
    api_key="EMPTY",
    base_url=VLLM_BASE_URL,
    max_tokens=4096,
    extra_body={
        "chat_template_kwargs": {"enable_thinking": False}
    },
)

In [9]:
chat.invoke("hello")

AIMessage(content='Okay, the user said "hello". I need to respond appropriately. Let me think.\n\nFirst, I should acknowledge their greeting. A simple "Hello!" would be good. Then, maybe ask how I can help them. But the user might not have a specific question yet. So, I should keep it friendly and open-ended.\n\nWait, in previous interactions, the assistant might have a standard response. Let me check. Oh, right, the standard is to say "Hello! How can I assist you today?" or something similar.\n\nBut maybe the user is just testing if the chatbot is working. So, a warm and helpful response is key. I should make sure to be polite and offer assistance. Let me phrase it: "Hello! How can I assist you today?" That\'s straightforward and invites the user to ask their question. I don\'t need to add anything else unless the user responds with a follow-up. So the response should be concise and friendly.\n</think>\n\nHello! How can I assist you today? 😊', additional_kwargs={'refusal': None}, resp

In [10]:
from langchain_classic.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [detect_object, segment_object, inpaint_image]
agent = create_tool_calling_agent(chat, tools=tools, prompt=prompt)

In [12]:
from langchain_classic.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [13]:
user_image = "dog.jpg"
user_text = "이 이미지에 있는 강아지를 우주복 입은 강아지로 바꿔줘."
prompt_for_agent = f"""당신은 이미지 편집 전문가입니다. 주어진 도구를 사용해서 사용자의 요청을 단계별로 처리해야 합니다.

# 사용 가능한 도구 목록:
- detect_object(image_path, object_name): 이미지에서 물체의 위치(좌표)를 찾습니다.
- segment_object(image_path, bounding_box_json): 좌표를 받아 마스크를 만듭니다.
- inpaint_image(image_path, mask_path, prompt): 마스크 영역을 프롬프트에 따라 수정합니다.

# 작업 절차:
1. 사용자의 요청을 분석해서 '무엇을' 찾을지 파악한 뒤, `detect_object`를 호출하세요.
2. `detect_object`의 결과(좌표)를 `segment_object`에 넘겨서 마스크를 만드세요.
3. 원본 이미지, 생성된 마스크, 그리고 사용자의 최종 요구사항을 담은 프롬프트를 `inpaint_image`에 넘겨서 최종 이미지를 만드세요.

# 사용자 요청사항
- 입력 이미지 경로: '{user_image}'
- 요청 내용: '{user_text}'

이제 위 절차에 따라 작업을 시작하고, 최종 결과물 파일 경로를 알려주세요.
"""

print("\n===== 에이전트 작업 시작! =====")


result = agent_executor.invoke({"input": prompt_for_agent})

print("\n===== 에이전트 작업 완료! =====")
print("최종 결과:")


===== 에이전트 작업 시작! =====


> Entering new AgentExecutor chain...

Invoking: `detect_object` with `{'image_path': 'dog.jpg', 'object_name': '강아지'}`
responded: Okay, let me break this down. The user wants to change the dog in the image 'dog.jpg' into a dog wearing a spacesuit. So first, I need to detect the dog in the image. The tool for that is detect_object. The parameters required are image_path and object_name. The image path is given as 'dog.jpg', and the object to detect is '강아지' (which is Korean for dog). So I'll call detect_object with those parameters.

Next step is to take the bounding box from the detect_object result and use segment_object to create a mask. The segment_object needs the image_path and the bounding_box_json. The image path is still 'dog.jpg', and the bounding box will come from the previous step's output.

Then, using inpaint_image, I need to replace the masked area with a spacesuit dog. The parameters here are image_path, mask_path, and prompt. The prompt shou